In [22]:
import pandas as pd
import numpy as numpy
import numpy as np

## Entrenamiento

In [23]:
tipos_conv = {'USER_ID':np.int32,
              'convirtio':np.bool,
              "cantidad_paginas_vistas_trimestre": np.int64,
             "cant_paginas_vistas_en_mes_1": np.int64,
             "cant_paginas_vistas_en_mes_2": np.int64,
              "cant_paginas_vistas_en_mes_3": np.int64}

In [24]:
train=pd.read_csv("../data/train_completo.csv")#,dtype=tipos_conv)

In [25]:
train.head()

,Unnamed: 0,USER_ID,convirtio,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,0,6574,False,1141.0,524.0,364.0,253.0
1,1,2639,True,671.0,261.0,179.0,231.0
2,2,3898,False,219.0,88.0,50.0,81.0
3,3,7996,False,383.0,95.0,178.0,110.0
4,4,6111,False,425.0,208.0,93.0,124.0


In [26]:
Y=train['convirtio']
Y.head()

0    False
1     True
2    False
3    False
4    False
Name: convirtio, dtype: bool

In [6]:
X=train[['cantidad_paginas_vistas_trimestre','cant_paginas_vistas_en_mes_1','cant_paginas_vistas_en_mes_2','cant_paginas_vistas_en_mes_3']]
X.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,1141.0,524.0,364.0,253.0
1,671.0,261.0,179.0,231.0
2,219.0,88.0,50.0,81.0
3,383.0,95.0,178.0,110.0
4,425.0,208.0,93.0,124.0


In [7]:
X=X.fillna(0)
X=X.astype(int)
X.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,1141,524,364,253
1,671,261,179,231
2,219,88,50,81
3,383,95,178,110
4,425,208,93,124


In [8]:
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

In [9]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(objective="binary:logistic", random_state=42)

In [10]:
xgb_model.fit(X, Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [11]:

#y_pred = xgb_model.predict(X)
#print(confusion_matrix(Y, y_pred))

In [12]:
xgb_model.score(X,Y)

0.9669978303071828

## Prediccion

In [13]:
predecir=pd.read_csv("../data/sampleSubmission.csv")#,dtype=tipos_conv)

In [14]:
predecir_pageviews=pd.read_csv("../data/features/predecir_pageviews.csv")

In [15]:
predecir_pageviews.head()

,Unnamed: 0,USER_ID,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,0,6574,1290,NaN,NaN,NaN
1,1,2639,836,NaN,NaN,NaN
2,2,3898,530,NaN,NaN,NaN
3,3,7996,150,NaN,NaN,NaN
4,4,6111,323,NaN,NaN,NaN


In [16]:
predecir_final = predecir.merge(predecir_pageviews,on = "USER_ID", how = 'left')

In [17]:
predecir_final.head()

,USER_ID,SCORE,Unnamed: 0,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,0,0,808,1519,NaN,NaN,NaN
1,1,0,1609,477,NaN,NaN,NaN
2,2,0,3136,651,NaN,NaN,NaN
3,3,0,5544,381,NaN,NaN,NaN
4,4,0,4273,627,NaN,NaN,NaN


In [18]:
predecir_final=predecir_final[['cantidad_paginas_vistas_trimestre','cant_paginas_vistas_en_mes_1','cant_paginas_vistas_en_mes_2','cant_paginas_vistas_en_mes_3']]
predecir_final=predecir_final.fillna(0)
predecir_final=predecir_final.astype(int)
predecir_final.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
0,1519,0,0,0
1,477,0,0,0
2,651,0,0,0
3,381,0,0,0
4,627,0,0,0


In [19]:
y_predict=xgb_model.predict(predecir_final)

In [20]:
predecir['SCORE']=y_predict

In [21]:
predecir.set_index('USER_ID',inplace=True)
predecir.head()

,SCORE
USER_ID,
0,False
1,False
2,False
3,False
4,False


In [28]:
predecir.to_csv('../data/submit_xgb.csv')